In [ ]:
# %mkdir ./data
# !wget -O ./data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -zxf ./data/aclImdb_v1.tar.gz -C ./data

In [ ]:
import os
import glob

def read_imdb_data(data_dir='./data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [ ]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

In [ ]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [ ]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    # TODO: try out SnowBallStemmer
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [ ]:
import pickle

cache_dir = os.path.join("./cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [ ]:
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

In [ ]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    for sentence in data:
        for word in sentence:
            if word in word_count.keys():
                word_count[word] = word_count[word] + 1
            else:
                word_count[word] = 1
   
    sorted_words = {k: v for k, v in sorted(word_count.items(), key=lambda item: item[1], reverse=True)}
    sorted_words = [word for word in sorted_words.keys()]
    word_dict = {} # This is what we are building, a dictionary that translates words into integers

    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [ ]:
train_X, test_X, train_y, test_y = preprocess_data()
type(train_X), type(train_y)
train_X.extend(test_X)
train_y.extend(test_y)
len(train_X), len(train_y)

In [ ]:
import random
type(train_X), type(train_y)
temp = list(zip(train_X, train_y))
random.shuffle(temp)
train_X, train_y = zip(*temp)
# res1 and res2 come out as tuples, and so must be converted to lists.
train_X, train_y = list(train_X), list(train_y)

In [ ]:

type(train_X), type(train_y)
print(len(train_X))
train_list = train_X[:30000]
train_labels_list = train_y[:30000]
test_list = train_X[30000:]
test_labels_list = train_y[30000:]
print(len(train_list), len(train_labels_list), len(test_list), len(test_labels_list))
print(train_list[0], test_list[0], train_labels_list[0], test_labels_list[0])

In [ ]:
word_dict = build_dict(train_list)

In [ ]:
data_dir = './data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [ ]:
with open(os.path.join(data_dir, 'word_dict_qrnn.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

In [ ]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_list)
test_X, test_X_len = convert_and_pad_data(word_dict, test_list)


In [ ]:
print(train_X.shape, len(train_X_len))
print(test_X.shape, len(test_X_len))
print(train_y[0])

In [ ]:
import pandas as pd
pd.concat([pd.DataFrame(train_labels_list), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1).to_csv(os.path.join(data_dir, 'train_qrnn.csv'), header=False, index=False)

In [ ]:
pd.concat([pd.DataFrame(test_labels_list), pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1).to_csv(os.path.join(data_dir, 'test_qrnn.csv'), header=False, index=False)

# If Files present start from here

In [1]:
import pandas as pd
import os
import pickle

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_qrnn.pkl'), "rb") as f:
    word_dict = pickle.load(f)

In [2]:
import pandas as pd
import os
train = pd.read_csv(os.path.join(data_dir, 'train_qrnn.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_qrnn.csv'), header=None, names=None)
print(train.shape, test_sample.shape)

(30000, 502) (20000, 502)


In [3]:
from sklearn.model_selection import train_test_split
test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape

((30000, 502), (10000, 502), (10000, 502))

In [4]:
import torch
import torch.utils.data

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float().squeeze()
train_X = torch.from_numpy(train.drop([0], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float().squeeze()
val_X = torch.from_numpy(val.drop([0], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float().squeeze()
test_X = torch.from_numpy(test.drop([0], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)

torch.Size([10000])


In [5]:
import torch.nn as nn
import numpy as np

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig(out.squeeze())

In [6]:
# write to file.
filename = "LSTM_imdb.csv"
def write_to_csv(epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [9]:
import time
def train(model, train_dl, val_dl, epochs, optimizer, loss_fn, device):
    train_loss_epoch = []
    train_acc_epoch = []
    val_loss_epoch = []
    val_accuracy_epoch = []
    time_train = []
    for epoch in range(epochs):
        model.train()
        start = time.time()
        total_loss = 0
        train_acc = 0
        total = 0
        correct = 0
        for batch in train_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            optimizer.zero_grad()
            prediction = model(batch_X)
            loss = loss_fn(prediction, batch_y)
            loss.backward()
            optimizer.step()
            result = np.round(prediction.detach().cpu())
            total_loss += loss.data.item()
            total += batch_y.size(0)
            correct += (result == batch_y.cpu()).sum().item()
            train_acc = correct/total
        train_loss_epoch.append(np.round(total_loss / len(train_dl), 3))
        train_acc_epoch.append(np.round(train_acc*100,3))
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_dl)))
        
        with torch.no_grad():
            model.eval()
            correct = 0
            total = 0
            val_loss = []
            for inputs, labels in val_dl:
                inputs_val, labels_val = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                prediction = model(inputs_val)
                loss = loss_fn(prediction, labels_val)
                val_loss.append(np.round(loss.item(),3))
                result = np.round(prediction.cpu())
                total += labels_val.size(0)
                correct += (result == labels_val.cpu()).sum().item()
            val_accuracy_epoch.append(np.round((correct/total)*100, 3))
            val_loss_epoch.append(np.round(np.mean(val_loss),3))
            end = time.time() - start
            print("Val Loss: {:.3f}".format(np.mean(val_loss)), "\tVal Acc: {:.3f}".format(correct/total))
            time_train.append(np.round(end,3))
    write_to_csv(epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_accuracy_epoch, time_train)
    return model

def test(model, test_dl, epochs):
    model.eval()
    correct = 0
    total = 0
#     results = []
#     labels = []
    with torch.no_grad():
        for batch in test_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            prediction = model(batch_X)
            result = np.round(prediction.cpu())
#             results.extend(list(result.numpy()))
#             labels.extend(list(batch_y.numpy()))
            total += batch_y.size(0)
            correct += (result == batch_y).sum().item()
    acc = correct/total
    append_to_csv(epochs, np.round(acc*100, 3))
    print("Accuracy:", (correct/total)*100)

In [10]:
import torch.optim as optim
epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()
train(model, train_dl, val_dl, epochs, optimizer, loss_fn, device)

Epoch: 0, BCELoss: 0.601867962082227
Val Loss: 0.487 	Val Acc: 0.768
Epoch: 1, BCELoss: 0.41929418275753655
Val Loss: 0.399 	Val Acc: 0.829
Epoch: 2, BCELoss: 0.34020929778615633
Val Loss: 0.357 	Val Acc: 0.847
Epoch: 3, BCELoss: 0.35414703994989394
Val Loss: 0.397 	Val Acc: 0.829
Epoch: 4, BCELoss: 0.30965810439238944
Val Loss: 0.320 	Val Acc: 0.865
Epoch: 5, BCELoss: 0.28246908616274596
Val Loss: 0.321 	Val Acc: 0.869
Epoch: 6, BCELoss: 0.24996221296489238
Val Loss: 0.326 	Val Acc: 0.871
Epoch: 7, BCELoss: 0.239388638548553
Val Loss: 0.320 	Val Acc: 0.877
Epoch: 8, BCELoss: 0.22275431678940852
Val Loss: 0.328 	Val Acc: 0.880
Epoch: 9, BCELoss: 0.211343239210546
Val Loss: 0.384 	Val Acc: 0.867


LSTMClassifier(
  (embedding): Embedding(5000, 32, padding_idx=0)
  (lstm): LSTM(32, 100)
  (dense): Linear(in_features=100, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [11]:
test(model, test_dl, epochs)

Accuracy: 87.13
